In [ ]:

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seawater
import netCDF4 as nc
import sys
sys.path.append('/data/tjarniko/mocsy')
import mocsy
import gsw
import warnings


In [ ]:
df = pd.read_csv('./Datasets/2015dat_fromdf.csv')
dsetname = 'D15'

dic = df['dic'].to_numpy()
alk = df['alk'].to_numpy()
dic_qf = df['dic_qf'].to_numpy()
alk_qf = df['alk_qf'].to_numpy()
lat = df['lat'].to_numpy()
lon = df['lon'].to_numpy()
S = df['S'].to_numpy()
T = df['T'].to_numpy()
P = df['P'].to_numpy()
mon = df['mon'].to_numpy()
day = df['day'].to_numpy()




tzero = np.zeros_like(ta)
dens = seawater.dens(S,T,P)
# CONVERT FROM UMOL TO MMOL/M3
dic_conv=dic*dens/1000 * 1e-3
ta_conv = ta*dens/1000 * 1e-3
tpressure = tzero +1 

response_tup = mocsy.mvars(temp=T, sal=S, alk=ta_conv, dic=dic_conv,
                   sil=tzero, phos=tzero, patm=tpressure, depth=P, lat=tzero,
                    optcon='mol/m3', optt='Tinsitu', optp='m',
                    optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
pH,pco2,fco2,co2,hco3,co3,OmegaA_OBS,OmegaC,BetaD,DENis,p,Tis = response_tup